In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. Data Cleaning and Preparation
file_name = "StudentPerformanceFactors.csv"
df = pd.read_csv(file_name)

# Select columns needed for the unified model
selected_cols = [
    'Exam_Score', 'Hours_Studied', 'Attendance', 'Sleep_Hours',
    'Previous_Scores', 'Tutoring_Sessions', 'Physical_Activity',
    'Parental_Involvement', 'Teacher_Quality', 'Parental_Education_Level',
    'Gender'
]
data_clean = df[selected_cols].dropna()

print(f"Original Row Count: {len(df)}")
print(f"Cleaned Row Count (Complete Cases): {len(data_clean)}")
print("-" * 30)

# 2. Unified Model Formula with Interactions
# C() explicitly tells statsmodels to treat the variable as categorical
# * in 'C(Parental_Involvement) * Hours_Studied' includes both main effects and the interaction term
formula = (
    "Exam_Score ~ Hours_Studied + Attendance + Sleep_Hours + Previous_Scores + "
    "Tutoring_Sessions + Physical_Activity + "
    "C(Parental_Involvement) * Hours_Studied + "  # Key Interaction Term
    "C(Teacher_Quality) + C(Parental_Education_Level) + C(Gender)"
)

# 3. Multicollinearity Check (VIF)
# To calculate VIF, we need the design matrix (X) which contains the dummy variables
model_vif = ols(formula, data=data_clean).fit()
X = model_vif.model.exog
X_df = pd.DataFrame(X, columns=model_vif.model.exog_names)

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["Variable"] = X_df.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_df.values, i)
    for i in range(X_df.shape[1])
]

# Exclude the constant (Intercept) for VIF reporting
vif_data = vif_data[vif_data['Variable'] != 'Intercept'].sort_values(by='VIF', ascending=False)

print("\n--- Multicollinearity Check (VIF) ---")
print("VIF > 5 is often considered problematic. VIF > 10 is usually severe.")
print(vif_data)
print("-" * 30)


# 4. Robust GLM Fit
# Using cov_type='HC3' for Heteroscedasticity-Consistent (Robust) Standard Errors
robust_model = ols(formula, data=data_clean).fit(cov_type='HC3')

print("\n--- Robust General Linear Model Summary (Final Inference Model) ---")
print("Note: The P>|t| column uses Robust Standard Errors (HC3) for accurate inference.")
print(robust_model.summary())

Original Row Count: 6607
Cleaned Row Count (Complete Cases): 6443
------------------------------

--- Multicollinearity Check (VIF) ---
VIF > 5 is often considered problematic. VIF > 10 is usually severe.
                                           Variable        VIF
9      C(Parental_Involvement)[T.Low]:Hours_Studied  17.478572
10  C(Parental_Involvement)[T.Medium]:Hours_Studied  17.331085
1                    C(Parental_Involvement)[T.Low]  17.056262
2                 C(Parental_Involvement)[T.Medium]  15.874722
8                                     Hours_Studied   3.287416
6       C(Parental_Education_Level)[T.Postgraduate]   1.327791
5        C(Parental_Education_Level)[T.High School]   1.327716
3                         C(Teacher_Quality)[T.Low]   1.206319
4                      C(Teacher_Quality)[T.Medium]   1.205430
13                                  Previous_Scores   1.003875
15                                Physical_Activity   1.003003
11                                     